In [0]:
-- ================================
-- ECO TRACK PROJECT
-- Medallion Architecture (Batch)
-- Databricks Free Edition
-- ================================

-- ================================
-- SET CATALOG & SCHEMA
-- ================================
USE CATALOG ecotrack;
USE SCHEMA ecotrack_schema;

-- ================================
-- BRONZE LAYER
-- ================================

DROP TABLE IF EXISTS bronze_air_quality_raw;

CREATE TABLE bronze_air_quality_raw
USING DELTA
AS
SELECT *
FROM csv.`/Volumes/ecotrack/ecotrack_schema/ecotrack_dataset/city_day.csv`;

-- ================================
-- SILVER LAYER
-- ================================

DROP TABLE IF EXISTS silver_air_quality_clean;

CREATE TABLE silver_air_quality_clean
USING DELTA
AS
SELECT
  _c0 AS city,
  TRY_CAST(_c1 AS DATE)   AS date,
  TRY_CAST(_c2 AS DOUBLE) AS pm25,
  TRY_CAST(_c3 AS DOUBLE) AS pm10,
  TRY_CAST(_c5 AS DOUBLE) AS no2,
  TRY_CAST(_c8 AS DOUBLE) AS co,
  TRY_CAST(_c9 AS DOUBLE) AS so2,
  TRY_CAST(_c10 AS DOUBLE) AS o3,
  TRY_CAST(_c13 AS INT)   AS aqi,
  _c14 AS aqi_bucket
FROM bronze_air_quality_raw
WHERE _c0 <> 'City'
  AND TRY_CAST(_c1 AS DATE) IS NOT NULL;

-- ================================
-- GOLD LAYER
-- ================================

-- 1️ City AQI Summary
DROP TABLE IF EXISTS gold_city_aqi_summary;

CREATE TABLE gold_city_aqi_summary
USING DELTA
AS
SELECT
  city,
  COUNT(*) AS total_days,
  ROUND(AVG(pm25), 2) AS avg_pm25,
  ROUND(AVG(pm10), 2) AS avg_pm10,
  MAX(aqi) AS max_aqi,
  MIN(aqi) AS min_aqi
FROM silver_air_quality_clean
GROUP BY city;

-- 2️ AQI Bucket Summary
DROP TABLE IF EXISTS gold_aqi_bucket_summary;

CREATE TABLE gold_aqi_bucket_summary
USING DELTA
AS
SELECT
  city,
  aqi_bucket,
  COUNT(*) AS days_count
FROM silver_air_quality_clean
GROUP BY city, aqi_bucket;

-- 3️ High Risk Cities
DROP TABLE IF EXISTS gold_high_risk_cities;

CREATE TABLE gold_high_risk_cities
USING DELTA
AS
SELECT
  city,
  COUNT(*) AS severe_days
FROM silver_air_quality_clean
WHERE aqi_bucket IN ('Poor', 'Very Poor', 'Severe')
GROUP BY city
HAVING COUNT(*) > 10;

-- 4 Monthly Pollution Trend
DROP TABLE IF EXISTS gold_monthly_pollution_trend;

CREATE TABLE gold_monthly_pollution_trend
USING DELTA
AS
SELECT
  city,
  YEAR(date)  AS year,
  MONTH(date) AS month,
  ROUND(AVG(pm25), 2) AS avg_pm25,
  ROUND(AVG(pm10), 2) AS avg_pm10
FROM silver_air_quality_clean
GROUP BY city, YEAR(date), MONTH(date);
